<a href="https://colab.research.google.com/github/ekaterinatao/NER_biomed_domain/blob/main/active_learning/%D0%92%D0%9A%D0%A0_nerel_bio_RuBioBERT_active_learning_v3_acc_seeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение с разными сидами для расчета доверительных интервалов
# Инструменты
Предобработанный дасасет [NEREL-BIO](https://huggingface.co/datasets/ekaterinatao/nerel_bio_ner_unnested)  

Исходная модель [RuBioBERT](https://huggingface.co/alexyalunin/RuBioBERT)

### Установка зависимостей

In [1]:
!pip install datasets accelerate evaluate wandb seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import random
import json
from dataclasses import dataclass
import torch
from torch.nn.functional import softmax
import datasets
from datasets import Dataset, DatasetDict, concatenate_datasets
from transformers import (AutoTokenizer,
                          DataCollatorForTokenClassification,
                          AutoModelForTokenClassification,
                          TrainingArguments, Trainer)
import evaluate
import warnings
warnings.filterwarnings("ignore")

In [3]:
@dataclass
class TrainingConfig:
    dataset = 'ekaterinatao/nerel_bio_ner_unnested'
    checkpoint = 'alexyalunin/RuBioBERT'
    hf_repo_id = "ekaterinatao/nerel-bio-RuBioBERT-al"
    n_labels = 45
    n_epochs = 10
    train_batch_size = 4
    eval_batch_size = 4
    device = "cuda" if torch.cuda.is_available() else "cpu"
    l_rate = 5e-05
    w_decay = 0.1
    warm_up = 0.1
    run_name = "RuBioBERT_AL_v3_3"

config = TrainingConfig()

In [4]:
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("Current seed: ", seed)

In [5]:
# генерация фиксированного списка сидов для воспроизводимости экспериментов
np.random.seed(64)
seed_lst = np.random.randint(65, 2024, size=4)

### Скачивание датасета

In [6]:
dataset = datasets.load_dataset(config.dataset)
dataset

Generating train split:   0%|          | 0/612 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/77 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/77 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 612
    })
    valid: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 77
    })
    test: Dataset({
        features: ['id', 'words', 'ner_tags'],
        num_rows: 77
    })
})

In [7]:
# Labels
url = 'https://raw.githubusercontent.com/ekaterinatao/NER_biomed_domain/main/labels.txt'
tags = pd.read_csv(url, names=['tag']).values.tolist()
tags = [item for sublist in tags for item in sublist]
tag_to_id = {tag: i for i, tag in enumerate(tags)}
id_to_tag = {i: tag for i, tag in enumerate(tags)}

___
### Токенизация

In [8]:
tokenizer = AutoTokenizer.from_pretrained(config.checkpoint)

tokenizer_config.json:   0%|          | 0.00/413 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["words"], truncation=True, max_length=512, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_idxs = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_idxs:
            if word_idx is None:
                label_ids.append(-100) # Set the special tokens to -100.
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [10]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels, batched=True
)
tokenized_dataset

Map:   0%|          | 0/612 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 612
    })
    valid: Dataset({
        features: ['id', 'words', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 77
    })
    test: Dataset({
        features: ['id', 'words', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 77
    })
})

# Обучение модели

In [11]:
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [tags[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tags[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [12]:
def choose_samples_accuracy(test_predictions, n_tokens, test_dataset) -> np.ndarray:
    """Функция для реализации стратеги отбора примеров для разметки
    на основе доли правильно предсказанных тэгов"""

    predictions = np.argmax(test_predictions, axis=2)
    pred_tags = [
        [tags[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, test_dataset['labels'])
    ]
    real_tags = [
        [tags[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, test_dataset['labels'])
    ]
    scores = [
        sum([p == l for p, l in zip(prediction, label)]) / len(label)
        for prediction, label in zip(pred_tags, real_tags)
    ]
    args = np.argsort(scores)

    return args[:n_tokens]

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [14]:
print(f'device is {config.device}')

device is cuda


In [15]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
def train_and_predict(
    al_train_dataset, eval_dataset, al_test_dataset, test_dataset,
    model, tokenizer, data_collator, compute_metrics, SEED
):
    "Функция для тренировки модели, сохранения метрик и предсказаний"

    global config
    training_args = TrainingArguments(
        output_dir=config.hf_repo_id,
        num_train_epochs=config.n_epochs,
        learning_rate=config.l_rate,
        weight_decay=config.w_decay,
        warmup_ratio=config.warm_up,
        per_device_train_batch_size=config.train_batch_size,
        per_device_eval_batch_size=config.eval_batch_size,
        group_by_length=True,
        optim="adamw_torch",
        lr_scheduler_type="cosine",
        evaluation_strategy="epoch",
        seed = SEED,
        data_seed = SEED,
        push_to_hub=True,
        save_strategy="no",
        report_to="wandb",
        logging_steps=16,
        run_name=config.run_name + f'{SEED}',
    )
    metrics = {}
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=al_train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    train_result = trainer.train()
    metrics['train_loss'] = train_result.metrics['train_loss']
    metrics.update(trainer.evaluate(eval_dataset))
    if al_test_dataset.num_rows > 0:
        al_test_predictions = trainer.predict(
            al_test_dataset, metric_key_prefix="pred"
        )
        probas = al_test_predictions.predictions
        metrics.update(al_test_predictions.metrics)
    else:
        probas = None
    test_predictions = trainer.predict(test_dataset, metric_key_prefix="test")
    metrics.update(test_predictions.metrics)

    return metrics, probas, trainer

In [18]:
def run_active_learning(
    dataset, al_iters, init_train_size, choose_tokens,
    tokenizer, data_collator, compute_metrics, SEED
):
    "Функция для реализации цикла активного обучения"

    train_dataset = dataset['train'].select(
        random.sample(
            range(dataset['train'].num_rows),
            int(dataset['train'].num_rows * init_train_size)
        )
    )
    test_dataset = dataset['train'].filter(
        lambda s: s['id'] not in train_dataset['id']
    )
    all_metrics = {}

    for iter in range(al_iters):
        train_size = train_dataset.num_rows
        print(f'\nIteration: {iter}. Training using {train_size} samples')
        model = AutoModelForTokenClassification.from_pretrained(
            config.checkpoint, num_labels=config.n_labels,
            id2label=id_to_tag, label2id=tag_to_id
        )

        metrics, test_predictions, trainer = train_and_predict(
            train_dataset, dataset['valid'], test_dataset, dataset['test'],
            model, tokenizer, data_collator, compute_metrics, SEED
        )

        all_metrics[f'{iter}'] = {'train_size': train_size, 'metrics': metrics}

        if test_dataset.num_rows >= choose_tokens:
            assert test_predictions is not None, "test_predictions is None, must be array"
            new_args = choose_samples_accuracy(test_predictions, choose_tokens, test_dataset)
            new_train_samples = test_dataset.select(new_args)
            extended_train_dataset = concatenate_datasets(
                [train_dataset, new_train_samples]
            )

            train_dataset = extended_train_dataset
            test_dataset = dataset['train'].filter(
                lambda s: s['id'] not in train_dataset['id']
            )

    return all_metrics, trainer

# Сэмплирование по 4 примера (по ~0.7% данных, 30 итераций активного обучения) для каждого сида (4 цикла).

In [19]:
for SEED in seed_lst:
    fix_seed(SEED)
    run = wandb.init(
        project="ner_bert_nerel_bio",
        name=config.run_name + f'{SEED}',
        reinit=True)
    metrics, trainer = run_active_learning(
        dataset=tokenized_dataset,
        al_iters=30,
        init_train_size=0.007,
        choose_tokens=4,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        SEED=SEED
    )
    run.join()
    with open(f'/content/drive/MyDrive/data/VKR/metrics_30_accuracy_seed{SEED}.json', 'w') as file:
        json.dump(metrics, file)

wandb: Currently logged in as: taoea. Use `wandb login --relogin` to force relogin


Current seed:  517


Filter:   0%|          | 0/612 [00:00<?, ? examples/s]


Iteration: 0. Training using 4 samples


config.json:   0%|          | 0.00/889 [00:00<?, ?B/s]

...


Iteration: 29. Training using 120 samples


Some weights of BertForTokenClassification were not initialized from the model checkpoint at alexyalunin/RuBioBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,3.594800,2.533066,0.323407,0.272514,0.295787,0.477234
2,2.170600,1.292322,0.638460,0.610460,0.624146,0.728827
3,1.164700,0.914826,0.718949,0.686914,0.702567,0.795338
4,0.719400,0.783150,0.731910,0.723499,0.727680,0.814608
5,0.536400,0.721671,0.750653,0.741323,0.745959,0.827506
6,0.345800,0.690692,0.761003,0.754221,0.757597,0.833566
7,0.282400,0.696390,0.770789,0.758677,0.764685,0.838539
8,0.224600,0.702194,0.770581,0.761726,0.766128,0.839161
9,0.208800,0.708794,0.768319,0.759850,0.764061,0.838850
10,0.182500,0.711167,0.768994,0.761961,0.765461,0.839938


Filter:   0%|          | 0/612 [00:00<?, ? examples/s]

eval/accuracy,▂▁▅▅▄▆▇▆▃▇▇▆▂▅██▇▂██▆███▅████▆███▃██▇███
eval/f1,▁▁▄▄▂▅▆▅▂▆▇▆▁▄▇▇▇▁▇█▆██▇▄████▆███▂██▇███
eval/loss,▇█▆▅▆▄▃▄▆▃▂▃▇▅▂▂▃▆▁▁▃▁▁▁▄▁▁▁▁▃▁▁▁▆▁▁▂▁▁▁
eval/precision,▂▁▄▄▃▅▆▅▃▆▇▆▂▄▇▇▇▂██▆██▇▅████▆███▃██▇███
eval/recall,▁▁▄▄▂▄▆▅▂▆▆▅▁▃▇▇▆▁▇▇▅██▇▄████▅██▇▂██▇███
eval/runtime,▁▄▂▁▂▁▁█▁▁▂▁▃▂▇█▄▂▁▃█▂▂▅▁▂▁▇▁██▁▁▁▂▂▂▂▁▁
eval/samples_per_second,█▅▇█▇▇█▁██▇█▅▇▂▁▄▇▇▆▁▇▇▄█▇█▂▇▁▁███▇▇▇▇▇█
eval/steps_per_second,█▅▇█▇▇█▁██▇█▅▇▂▁▄▇▇▆▁▇▇▄█▇█▂▇▁▁███▇▇▇▇▇█
train/epoch,▆█▂▃▃▄▃▃▃▁█▇▆▄▃█▆▄▂█▅▃█▅▂▇▄▁▆▃▇▄█▅▂▆▂▇▃▇
train/global_step,▁▁▁▁▁▂▂▂▂▁▄▄▃▃▂▅▄▃▂▅▄▂▆▄▂▆▄▁▅▃▇▄█▅▂▆▃▇▃█
train/grad_norm,▂▃▅▃▄▂▄▃▄▃▆▄▄▆▅▂▄▁▄▃▅▁▃▄█▂▄▂▄▁▆▄▂▃▁▇▃▄▃▂
